In [8]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://99df69d04a18:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [9]:
from spark_utils import SparkUtils

In [10]:
schema = [
    ("from", "string"),
    ("to", "string")
]
spark_schema = SparkUtils.generate_schema(schema)
#i know this is not a good practice when it comes to pyspark i shoudnt direct to a single document, 
# more so when its supposed to take a lot more than just one document
df = spark.read.option("header", True).schema(spark_schema).csv("/opt/spark/work-dir/data/connections.csv")

string
string
